In [1]:
from numpy import unique
from pandas import read_csv
url = 'https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv'
df = read_csv(url)
df.head()


,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [3]:
#get the first 10 columns
df_len_10 = df['text'].iloc[:100]


In [4]:
#CLASSES
import random, math
import io
from itertools import chain
import pandas as pd

#QUESTION 1
class Shingling:
   
    def __init__(self, input_text, input_k):
        self.text = input_text
        self.k = input_k
           

    def shingles(self):
        for line in self.text:
            return {line[i:i + self.k] for i in range(len(line) - self.k + 1)} 
    
    
    def ordered_hash(self, input_sh):
        random.seed(111)
        a = random.randint(0,100)
        b = random.randint(0,100)        #convert string into int
        return sorted(set((a*(int.from_bytes(x.encode(), 'little')) + 
                           b)%1039205197 for x in input_sh))
    
#QUESTION 2  
'''                
class CompareSets:
   
    @staticmethod
    def Jaccard(set1, set2):
        #return len(set1)/len(set2)
        return len(set1.intersection(set2)) / len(set1.union(set2))
'''    
class CompareSetsInput:
    def __init__(self, set1, set2):
        self.seta = set1
        self.setb = set2
    
    def Jac_Sim(self):
        return len(self.seta.intersection(self.setb)) / len(self.seta.union(self.setb))
    

#Question 3
class MinHashing:

    def __init__(self, input_set, input_n):
        self.s = input_set
        self.n = input_n

    def hashing(self):
        min_hash = [None] * self.n # Vector of length n
        random.seed(111)
        for n in range(self.n):
            a = random.randint(0, 2**32-1) # Max shingle id
            b = random.randint(0, 2**32-1)
            vector = [None] * len(self.s) # Vector of length set hashed shingles
            for i, value in enumerate(self.s):
                hashcode = ((a * value + b) % 4294967311) # Make sure that prime > max shingle id
                vector[i] = hashcode 
            min_hash[n] = min(vector)
        return min_hash
    
#Question 4    
class CompareSignatures:
    def __init__(self, vector1, vector2):
        self.v1 = vector1
        self.v2 = vector2

    def estimate(self):
        count = 0
        for i, elem in enumerate(self.v1):
            if self.v1[i] == self.v2[i]: # Assumed that length of both vectors are the same
                count += 1

        return count/len(self.v1)
    
#Qustion 5 

class LHS:
    def __init__(self, signatures, n, threshold):
        self.signatures = signatures
        self.n = n
        self.threshold = threshold
    
     
    def candidates(self):
        b, total_rows, total_docs = self.n, self.signatures.count().min(), len(self.signatures.columns)
        r = math.floor(total_rows / b)
        
        bucket = {} #dictionary
        cand = []
        
        for i in range(b):
            band = self.signatures[r*i:r*(i+1)]   #bands
            
            for j in range(total_docs):
                bucket[j] = hash(tuple(band[j].values)) #hash content of bands-->same bands will have same hash value
           
                
            for f in range(total_docs):
                for s in range(f+1, total_docs):
                    if bucket[f]==bucket[s]:
                        cand.append([f,s])    #get pairs of columns with same hash value (identical bands)
            
        self.final_cand = [list(x) for x in set(tuple(x) for x in cand)]      
        return(self.final_cand)
    
    def check_similarity(self):
        if self.final_cand:
            rows = self.signatures.count().min()
            for pair in self.final_cand:
                col1 = pair[0]
                col2 = pair[1]
                mh = []
                for i in [col1,col2]:
                    ordered_set2 = set(self.signatures[i].iloc[:rows])
                    mh.append(MinHashing(ordered_set2,20).hashing())

                if CompareSignatures(mh[0],mh[1]).estimate() <= self.threshold:
                    print("Column",col1,"and",col2,"have Jaccard similarity:",
                         CompareSignatures(mh[0],mh[1]).estimate(),"which is lower than",self.threshold,
                          "so they are not similar")
                else:        
                    print("Column",col1,"and",col2,"have Jaccard similarity:",
                          CompareSignatures(mh[0],mh[1]).estimate(),"which is higher than",self.threshold,"so they are similar")
        else:
            print("No candidates")

In [5]:
#get shingles and compute Jaccard similarity
text1 = io.StringIO(df_len_10[0])
text2 = io.StringIO(df_len_10[1])
sh1 = Shingling(text1,10)
shingles1 = sh1.shingles()
ordered = sh1.ordered_hash(shingles1)
print(len(ordered),"ordered=",ordered)
sh2 = Shingling(text2,10)
shingles2 = sh2.shingles()
Jac = CompareSetsInput(shingles1, shingles2)
Sim = Jac.Jac_Sim()
print(Sim)


4114 ordered= [179497, 478807, 915532, 955482, 1286611, 1910816, 2125669, 2402873, 2558635, 2564852, 2794174, 3322731, 3351782, 3440307, 3472416, 3871591, 4130470, 4515810, 4532477, 4616172, 4656476, 5250963, 5278691, 5797379, 6074874, 6171348, 6308576, 6619171, 6751311, 6828991, 7041556, 7082442, 7656722, 8148889, 8170852, 8194878, 8236213, 8280054, 8964004, 8991733, 9003669, 9053779, 9515933, 9648811, 10867830, 11068192, 11212727, 11321218, 11776869, 12209626, 12695187, 12866708, 12970642, 13291433, 13656608, 13682368, 13924290, 14092184, 14250210, 14260013, 14622631, 14646994, 14690000, 14691836, 14691890, 14708926, 15356706, 15572861, 15585152, 15662210, 16181796, 16376766, 16385724, 16392744, 16823152, 17281734, 17513221, 17556041, 17959355, 18287028, 18287325, 19488817, 19582736, 19930570, 19932005, 20271453, 20451933, 21088097, 21090846, 21328475, 21744849, 21822223, 21827830, 22095073, 22292228, 22779504, 23084179, 23195734, 23619038, 23839554, 23881513, 25140549, 25479890, 258

In [7]:
#q3
sample_text = io.StringIO(df_len_10[0])
# Construct 10-shingles
sh = Shingling(sample_text,10) 
shingles = sh.shingles() 
# Ordered set of hashed 100-shingles
ordered_set = sh.ordered_hash(shingles) 


mh = MinHashing(ordered_set,20)
minhash = mh.hashing()
print(minhash)

[2255858, 459508, 318683, 838973, 515248, 629171, 5330489, 707382, 2954840, 840384, 567677, 2533713, 408221, 540464, 1391777, 936275, 403876, 2043997, 3256226, 826186]


In [8]:
#q4
sets = []
for i in range(2):
    sample_text = io.StringIO(df_len_10[i])
    # Construct 10-shingles
    sh = Shingling(sample_text,10) 
    shingles = sh.shingles() 
    # Ordered set of hashed 100-shingles
    ordered_set = sh.ordered_hash(shingles)
    sets.append(ordered_set)


mh1 = MinHashing(sets[0],20).hashing()
mh2 = MinHashing(sets[1],20).hashing()
print(CompareSignatures(mh1,mh1).estimate())
print(CompareSignatures(mh1,mh2).estimate())

1.0
0.0


In [9]:
#input signatures in LHS
df = pd.DataFrame(sets).transpose()
df.head()

,0,1
0,179497.0,637954.0
1,478807.0,1555769.0
2,915532.0,1590398.0
3,955482.0,2261483.0
4,1286611.0,2497628.0


In [10]:
#q5 candidates and their similarity
l = LHS(df, 200, 0.8)
l.candidates()
l.check_similarity()

No candidates


In [11]:
import copy
df_with_duplicates = copy.deepcopy(df) 
df_with_duplicates[2] = df_with_duplicates[0]
df_with_duplicates[3] = df_with_duplicates[1]
print(df_with_duplicates.head())
print(df.head())

l_test = LHS(df_with_duplicates, 200, 0.8)
l_test.candidates()
l_test.check_similarity()

           0          1          2          3
0   179497.0   637954.0   179497.0   637954.0
1   478807.0  1555769.0   478807.0  1555769.0
2   915532.0  1590398.0   915532.0  1590398.0
3   955482.0  2261483.0   955482.0  2261483.0
4  1286611.0  2497628.0  1286611.0  2497628.0
           0          1
0   179497.0   637954.0
1   478807.0  1555769.0
2   915532.0  1590398.0
3   955482.0  2261483.0
4  1286611.0  2497628.0
Column 0 and 2 have Jaccard similarity: 1.0 which is higher than 0.8 so they are similar
Column 1 and 3 have Jaccard similarity: 1.0 which is higher than 0.8 so they are similar
